In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
%matplotlib inline

import nengo
import nengo_spa as spa

from nengo_learn_assoc_mem.utils import make_alt_vocab

# Apply in 2D first

In [ ]:
clust_vecs = 

## Negative Voja

In [ ]:
# make a model
# based off the reactions of the neurons, move the encoders towards the average of the last few activations

## Really slow Adaptive neurons and Voja

## Feedback alignment

# Apply to full vocab

In [4]:
dimensions = 64
n_neurons = 1000
seed = 8

vocab, fan1, fan1_vecs, fan2, fan2_vecs, foil1, foil1_vecs, foil2, foil2_vecs = make_alt_vocab(16, 16, dimensions, seed, norm=True)